# TSA Chapter 3: ARIMA Forecast for GDP

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch3/TSA_ch3_arima_forecast/TSA_ch3_arima_forecast.ipynb)

ARIMA(1,1,1) forecast with confidence intervals for US Real GDP.

In [ ]:
!pip install numpy pandas matplotlib statsmodels pandas-datareader -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import pandas_datareader as pdr
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
gdp = pdr.get_data_fred('GDPC1', start='1960-01-01', end='2024-09-30')
log_gdp = np.log(gdp['GDPC1'].dropna())

model = ARIMA(log_gdp, order=(1, 1, 1))
fit = model.fit()
fcast = fit.get_forecast(steps=8)
ci = fcast.conf_int()

fig, ax = plt.subplots(figsize=(10, 4))
recent = log_gdp.iloc[-40:]
ax.plot(recent.index, recent.values, color=BLUE, lw=1.5, label='Observed')
future = pd.date_range(start=log_gdp.index[-1], periods=9, freq='QS')[1:]
ax.plot(future, fcast.predicted_mean.values, color=RED, lw=1.5, ls='--', label='ARIMA(1,1,1) Forecast')
ax.fill_between(future, ci.iloc[:,0], ci.iloc[:,1], color=RED, alpha=0.15, label='95% CI')
ax.axvline(x=log_gdp.index[-1], color='black', ls=':', lw=1, alpha=0.5)
ax.set_title('ARIMA(1,1,1) Forecast: US Real GDP', fontweight='bold')
ax.set_ylabel('Log(GDP)')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)
plt.tight_layout()
save_chart(fig, 'ch3_arima_forecast')
plt.show()